In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error


path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-21 22:50:39,937 utils 400 [INFO]    [logger_func] start 


In [2]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(model_path):
    base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    tmp = utils.read_pkl_gzip(model_path)
    if 'pred_mean' in tmp.columns:
        pred_col = 'pred_mean'
    elif 'prediction' in tmp.columns:
        pred_col = 'prediction'
    tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
    
    train['pred'] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train['pred'].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================


#========================================================================
# 相関

def get_corr_ensemble(path, base, out_part = ''):
    '''
    1. ベースモデルとの相関を個別に取得する
    2. ベースモデルにpath_listのモデルの予測をjoinし、相関行列を取得する
    '''
    
    if key in base.columns:
        base = base.set_index(key)
    
    #========================================================================
    # 個別に相関を見ていく場合
    if str(type(path)).count('str'):
    
        if path.count('CV1'):
            return 0, ''
        if path.count('binary'):
            return 0, ''
        tmp = utils.read_pkl_gzip(path)
    
        try:
            tmp = tmp.reset_index()[[key, 'pred_mean']].set_index(key)
            base['tmp_pred'] = tmp.reset_index()['pred_mean']
        except KeyError:
            tmp = tmp.reset_index()[[key, 'prediction']].set_index(key)
            base['tmp_pred'] = tmp['prediction']
    
        if path.count('LB'):
            try:
                cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
            except AttributeError:
                return 0, ''
        else:
            cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
    
        corr = np.corrcoef(base['base_pred'], base['tmp_pred'].values).min()
    
        if corr>0 and corr<0.98:
            logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
            
            return corr, path
        else:
            return 0, ''
    #========================================================================
    
    #========================================================================
    # まとめて相関行列を見たい場合
    elif str(type(path)).count('list'):
        path_list = path
        for path in path_list:
        
            if out_part=='all':
                if path.count('CV-1') or path.count('binary'):
                    continue
            elif out_part=='rm_out':
                if path.count('CV-3') or path.count('binary'):
                    continue
            elif out_part=='binary':
                if path.count('CV-3') or path.count('CV-1'):
                    continue
            tmp = utils.read_pkl_gzip(path)

            if 'pred_mean' in tmp.columns:
                pred_col = 'pred_mean'
            elif 'prediction' in tmp.columns:
                pred_col = 'prediction'
                
            tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
                
            if path.count('LB'):
                try:
                    cv2 = re.search(r'CV([^/.]*)_LB', path.replace('.', '-')).group(1)
                except AttributeError:
                    print(path.replace('.', '-'))
                    continue
            else:
                cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
                
            base[f'pr_{cv2[:9]}'] = tmp[pred_col]
            
        if out_part=='rm_out':
            base = base[base[target]>-30]
        drop_cols = [col for col in base.columns if col.count('term') or col.count('no_out_flg') or col in ignore_list or col.count('clf')]
        base.drop(drop_cols, axis=1, inplace=True)
        base.sort_index(axis=1, inplace=True)
        mx_corr = base.corr(method='pearson')
#         mx_cor = mx_corr.drop(drop_cols, axis=1).loc[~mx_corr.index.isin(drop_cols)]
        
        return mx_corr
#========================================================================

In [20]:
is_clf = 0
base = utils.read_df_pkl('../input/base_no_out*')[[key, target, 'clf_pred', 'no_out_flg']].set_index(key)
# LB3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz').set_index(key)
# base['lb3664'] = LB3664['prediction']
#========================================================================
# rm_out以外の部分の相関を見る
# base = base[base['clf_pred']>=0.01]
#========================================================================
# ens_list = glob.glob('../stack/*lgb*all*CV3*.gz')
# ens_list = glob.glob('../model/LB3664_set/*_lgb_*.gz')
ens_list = glob.glob('../ensemble/lgb_ensemble/*_lgb_*.gz')
# ens_list = glob.glob('../ensemble/rm_outlier_ensemble/*_lgb_*.gz')
out_part = 'all'
# ens_list += glob.glob('../stack/0213_2*no_out*.gz') + glob.glob('../stack/0214_*no_out*.gz')

if is_clf:
    out_part = 'binary'
#     ens_list = glob.glob('../stack/0214*NN*-clf*.gz')
    ens_list = glob.glob('../ensemble/binary_ensemble/*.gz')
    base[target] = base[target].map(lambda x: 1 if x<-30 else 0)
top_corr = get_corr_ensemble(path=ens_list, base=base, out_part=out_part)
print(top_corr.shape)
display(top_corr.head())
# top_corr.to_csv('../output/0214_07_elo_rm_out_corr_ensemble_heatmap.csv', index=True)

100%|██████████| 1/1 [00:00<00:00,  4.05it/s]


(224, 224)


,pr_3-2381280,pr_3-2400031,pr_3-6179960,pr_3-6180025,pr_3-6180093,pr_3-6180576,pr_3-6180980,pr_3-6181163,pr_3-6181342,pr_3-6181697,pr_3-6181787,pr_3-6181994,pr_3-6182750,pr_3-6182807,pr_3-6183178,pr_3-6183913,pr_3-6184413,pr_3-6185842,pr_3-6186433,pr_3-6187062,pr_3-6187514,pr_3-6188437,pr_3-6188820,pr_3-6189458,pr_3-6189647,pr_3-6189733,pr_3-6190111,pr_3-6190564,pr_3-6191914,pr_3-6192567,pr_3-6192668,pr_3-6192899,pr_3-6196899,pr_3-6199406,pr_3-6200465,pr_3-6201274,pr_3-6201526,pr_3-6203135,pr_3-6203581,pr_3-6203797,pr_3-6204778,pr_3-6205796,pr_3-6206301,pr_3-6206456,pr_3-6206463,pr_3-6206848,pr_3-6207065,pr_3-6208082,pr_3-6208229,pr_3-6208407,pr_3-6208462,pr_3-6208488,pr_3-6208794,pr_3-6209578,pr_3-6209848,pr_3-6211455,pr_3-6212177,pr_3-6213553,pr_3-6214201,pr_3-6215750,pr_3-6216123,pr_3-6216744,pr_3-6216825,pr_3-6217349,pr_3-6218560,pr_3-6219266,pr_3-6219599,pr_3-6219900,pr_3-6219974,pr_3-6220296,pr_3-6221003,pr_3-6221122,pr_3-6221458,pr_3-6221631,pr_3-6221890,pr_3-6221915,pr_3-6223028,pr_3-6226545,pr_3-6227667,pr_3-6228204,pr_3-6229716,pr_3-6230340,pr_3-6232393,pr_3-6232415,pr_3-6232609,pr_3-6232696,pr_3-6232707,pr_3-6233222,pr_3-6233280,pr_3-6233788,pr_3-6236254,pr_3-6236411,pr_3-6236768,pr_3-6237199,pr_3-6237433,pr_3-6237892,pr_3-6238035,pr_3-6238109,pr_3-6238774,pr_3-6239445,...,pr_3-6283565,pr_3-6284144,pr_3-6284378,pr_3-6284657,pr_3-6286247,pr_3-6288632,pr_3-6290168,pr_3-6290897,pr_3-6296381,pr_3-6297296,pr_3-6297469,pr_3-6297689,pr_3-6311514,pr_3-6312489,pr_3-6314110,pr_3-6314888,pr_3-6315647,pr_3-6316576,pr_3-6319548,pr_3-6323019,pr_3-6323025,pr_3-6324662,pr_3-6327650,pr_3-6330951,pr_3-6333204,pr_3-6335126,pr_3-6335860,pr_3-6337832,pr_3-6341112,pr_3-6341660,pr_3-6358877,pr_3-6362052,pr_3-6364768,pr_3-6372746,pr_3-6377590,pr_3-6377968,pr_3-6378382,pr_3-6382497,pr_3-6382779,pr_3-6386524,pr_3-6388111,pr_3-6389269,pr_3-6392122,pr_3-6393394,pr_3-6394726,pr_3-6398254,pr_3-6401138,pr_3-6402002,pr_3-6403497,pr_3-6405123,pr_3-6406114,pr_3-6406494,pr_3-6406958,pr_3-6407080,pr_3-6410169,pr_3-6411694,pr_3-6412766,pr_3-6413082,pr_3-6413273,pr_3-6413337,pr_3-6413828,pr_3-6414174,pr_3-6415790,pr_3-6416975,pr_3-6420300,pr_3-6420397,pr_3-6420782,pr_3-6426586,pr_3-6427955,pr_3-6430089,pr_3-6431931,pr_3-6433852,pr_3-6434449,pr_3-6435566,pr_3-6437143,pr_3-6437365,pr_3-6438459,pr_3-6439138,pr_3-6439447,pr_3-6439522,pr_3-6439834,pr_3-6445879,pr_3-6450968,pr_3-6453887,pr_3-6462905,pr_3-6467847,pr_3-6468062,pr_3-6470229,pr_3-6476829,pr_3-6478281,pr_3-6493760,pr_3-6514559,pr_3-6536747,pr_3-6557007,pr_3-6561532,pr_3-6569439,pr_3-6574095,pr_3-6585232,pr_3-6594245,pr_3-9313964
pr_3-2381280,1.000000,0.984552,0.952858,0.949649,0.949531,0.950399,0.947248,0.948552,0.948357,0.948890,0.949959,0.947489,0.949900,0.948554,0.948033,0.948402,0.947796,0.949785,0.948962,0.949589,0.949155,0.950953,0.950230,0.949953,0.950082,0.949098,0.950162,0.949215,0.951184,0.948935,0.947515,0.949554,0.962418,0.963303,0.963070,0.962447,0.962360,0.961821,0.951777,0.962399,0.963731,0.963497,0.963313,0.950270,0.958261,0.962573,0.963409,0.962485,0.963190,0.962885,0.962037,0.963270,0.962616,0.950169,0.962530,0.962476,0.962957,0.953133,0.951247,0.961246,0.960805,0.954465,0.960139,0.961756,0.961159,0.962766,0.960698,0.961563,0.961882,0.961054,0.950571,0.960205,0.959896,0.961197,0.948359,0.961603,0.961402,0.948427,0.960233,0.958390,0.958284,0.958574,0.957792,0.958236,0.960170,0.958457,0.960128,0.958308,0.958481,0.957897,0.949438,0.958118,0.959271,0.958008,0.958212,0.958902,0.961544,0.957312,0.958082,0.960478,...,0.961980,0.960180,0.962427,0.960446,0.961807,0.962352,0.959113,0.962605,0.959066,0.949348,0.959161,0.960707,0.956120,0.955279,0.958413,0.954736,0.959638,0.883370,0.952579,0.957970,0.950415,0.950771,0.950805,0.950685,0.942884,0.935625,0.952405,0.951051,0.951071,0.951905,0.930206,0.929512,0.931335,0.931838,0.934104,0.922670,0.930574,0.931882,0.926307,0.923514,0.935240,0.923778,0.923328,0.935715,0.938092,0.924383,0.933579,0.935378,0.925278,0.916982,0.941890,0.927530,0.929302,0

In [21]:
#========================================================================
# Get Low Corr Model For Ensemble
select_list = []
thres_corr = 0.980
# thres_corr = 0.985
base_model = top_corr.iloc[0].index[0]
select_list.append(base_model)

def ensemble_corr_checker(model):
    df_corr = top_corr.loc[model].reset_index()
    
    current_idx = list(top_corr.index).index(model)
    df_corr = df_corr.iloc[current_idx:, ]
    best_cv = 100
    best_corr = 0
    for _, rows in df_corr.iterrows():
        cv = rows[0]
        corr = rows[1]
        
        if cv not in select_list and corr<thres_corr:
            tmp = top_corr.loc[select_list, cv]
            cnt = tmp[tmp>=thres_corr].sum()
            if cnt==0:
                return cv, corr
    
    return '', 0

# cv, corr = ensemble_corr_checker(base_model)
# if corr>=thres_corr:
#     continue
    
is_no_stack = 0
is_loop = True
while is_loop:
    for model in select_list:    
        cv, corr = ensemble_corr_checker(base_model)
        if len(select_list)==1:
            if corr:
                select_list.append(cv)
        elif len(select_list)>1:
            cv, corr = ensemble_corr_checker(select_list[-1])
            if corr:
                select_list.append(cv)
            else:
                is_loop=False
#========================================================================

In [22]:
print(select_list)
df_select = top_corr.loc[select_list, select_list]
# tmp = df_select['lb3664'].iloc[:9]
# select_list = list(tmp.index)
# select_list
df_select

['pr_3-2381280', 'pr_3-6179960', 'pr_3-6213553', 'pr_3-6216744', 'pr_3-6226545', 'pr_3-6236254', 'pr_3-6238109', 'pr_3-6251774', 'pr_3-6266191', 'pr_3-6297296', 'pr_3-6311514', 'pr_3-6316576', 'pr_3-6333204', 'pr_3-6358877', 'pr_3-6377968', 'pr_3-6388111', 'pr_3-6401138', 'pr_3-6406494', 'pr_3-6410169', 'pr_3-6416975', 'pr_3-6430089', 'pr_3-6431931', 'pr_3-6439138', 'pr_3-6450968', 'pr_3-6470229', 'pr_3-6478281', 'pr_3-6514559', 'pr_3-6536747', 'pr_3-6557007', 'pr_3-6569439']


,pr_3-2381280,pr_3-6179960,pr_3-6213553,pr_3-6216744,pr_3-6226545,pr_3-6236254,pr_3-6238109,pr_3-6251774,pr_3-6266191,pr_3-6297296,pr_3-6311514,pr_3-6316576,pr_3-6333204,pr_3-6358877,pr_3-6377968,pr_3-6388111,pr_3-6401138,pr_3-6406494,pr_3-6410169,pr_3-6416975,pr_3-6430089,pr_3-6431931,pr_3-6439138,pr_3-6450968,pr_3-6470229,pr_3-6478281,pr_3-6514559,pr_3-6536747,pr_3-6557007,pr_3-6569439
pr_3-2381280,1.000000,0.952858,0.953133,0.954465,0.948427,0.949438,0.957312,0.935781,0.951639,0.949348,0.956120,0.883370,0.942884,0.930206,0.922670,0.935240,0.933579,0.927530,0.922398,0.933719,0.931024,0.927631,0.930547,0.925226,0.923186,0.920419,0.924552,0.909176,0.909551,0.906974
pr_3-6179960,0.952858,1.000000,0.979459,0.974034,0.976253,0.968421,0.977964,0.958154,0.969951,0.963926,0.962482,0.900095,0.958873,0.959250,0.946939,0.945293,0.938788,0.934445,0.940576,0.945890,0.940339,0.935965,0.939144,0.930019,0.935595,0.936708,0.929450,0.916350,0.916819,0.909229
pr_3-6213553,0.953133,0.979459,1.000000,0.974969,0.978752,0.970044,0.979883,0.968332,0.970341,0.964098,0.961754,0.901060,0.959252,0.941976,0.931198,0.945605,0.938606,0.934256,0.927224,0.945922,0.939120,0.935704,0.929782,0.929974,0.923919,0.923803,0.928914,0.915531,0.907984,0.902523
pr_3-6216744,0.954465,0.974034,0.974969,1.000000,0.971371,0.967099,0.976205,0.955712,0.968803,0.965467,0.964337,0.899986,0.976818,0.945937,0.937378,0.949463,0.950540,0.940764,0.938673,0.949601,0.957258,0.946569,0.942328,0.941195,0.932089,0.932496,0.934354,0.922376,0.916230,0.915110
pr_3-6226545,0.948427,0.976253,0.978752,0.971371,1.000000,0.963689,0.973810,0.958321,0.964610,0.959614,0.955258,0.901517,0.955645,0.936095,0.924566,0.942162,0.934854,0.929987,0.921378,0.939925,0.935637,0.929066,0.922571,0.923939,0.919053,0.917869,0.923223,0.909561,0.900736,0.895563
pr_3-6236254,0.949438,0.968421,0.970044,0.967099,0.963689,1.000000,0.976233,0.950747,0.977181,0.974453,0.959956,0.894447,0.952421,0.941677,0.938009,0.945188,0.941169,0.939702,0.928922,0.944777,0.933042,0.942937,0.932605,0.931353,0.924368,0.930196,0.941723,0.918724,0.913026,0.904093
pr_3-6238109,0.957312,0.977964,0.979883,0.976205,0.973810,0.976233,1.000000,0.960273,0.969310,0.965017,0.978077,0.902950,0.961441,0.945761,0.944723,0.951039,0.952287,0.947959,0.939564,0.948445,0.940252,0.949342,0.944537,0.942067,0.931538,0.938088,0.945783,0.925517,0.923429,0.917523
pr_3-6251774,0.935781,0.958154,0.968332,0.955712,0.958321,0.950747,0.960273,1.000000,0.950982,0.945082,0.944970,0.958970,0.940733,0.923004,0.912738,0.928800,0.923127,0.917248,0.910904,0.927080,0.922626,0.917260,0.916312,0.913364,0.912620,0.909203,0.912489,0.898918,0.896762,0.893095
pr_3-6266191,0.951639,0.969951,0.970341,0.968803,0.964610,0.977181,0.969310,0.950982,1.000000,0.979311,0.963034,0.896862,0.955841,0.953501,0.940736,0.956629,0.945734,0.943413,0.934518,0.957608,0.945767,0.945880,0.937594,0.937120,0.935038,0.933784,0.936329,0.931608,0.916050,0.908463
pr_3-6297296,0.949348,0.963926,0.964098,0.965467,0.959614,0.974453,0.965017,0.945082,0.979311,1.000000,0.958513,0.891563,0.952950,0.948811,0.941235,0.950847,0.942711,0.943958,0.931169,0.952665,0.944478,0.945695,0.935201,0.933708,0.933298,0.934694,0.948957,0.922666,0.917810,0.907077


In [23]:
# ================================================================
# 相関行列から移動するpathを決める
import shutil
for path in ens_list:
    for sel in select_list:
#         sel_key = sel[3:].replace('-', '.')
        sel_key = sel[3:]
        if path.count(sel_key):
#             shutil.move(path, '../ensemble/lgb_ensemble/')
            try:
                shutil.move(path, '../ensemble/dir_stack_blend/')
#                 shutil.move(path, '../ensemble/rm_outlier_ensemble/tmp/')
            except FileNotFoundError:
                pass
            pass
        else:
            pass
#             shutil.move(path, '../ensemble/used_stack/')
# ================================================================

100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,target,clf_pred,no_out_flg
card_id,,,
C_ID_92a2005557,-0.820283,0.000444,1.0
C_ID_3d0044924f,0.392913,0.007831,0.0
C_ID_d639edf6cd,0.688056,0.004074,0.0
C_ID_186d6a6901,0.142495,0.000797,0.0
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0
